Se revisaron pappers para investigar los features que se suelen usar en el estado del arte.<br>
Los pappers seleccionados fueron los siguientes:<br>
<ul>
    <li>Prediction of plant lncRNA by ensemble machine learning classifiers</li>
    <li>LncADeep- an ab initio lncRNA identification and functional annotation tool based on deep learning</li>
    <li>A deep recurrent neural network discovers complex biological rules to decipher RNA protein-coding potential</li>
    <li>A Support Vector Machine based method to distinguish long non-coding RNAs from protein coding transcripts</li>
</ul>
<h1>1.Prediction of plant lncRNA by ensemble machine learning classifiers</h1>
Los features utilizados en este papper son:
<ol>
    <li>Longitud del transcriptoma</li>
    <li>Longitud del open reading frame (ORF)</li>
    <li>Porcentaje de nucleótidos GC presentes</li>
    <li>Fickett score</li>
    <li>Hexamer score</li>
    <li>Alignment identity in SwissProt database</li>
    <li>Longitud del alineamiento en la base de datos SwissProt</li>
    <li>Proporción de la longitud del alineamiento con respecto a la longitud del transcriptoma</li>
    <li>Proporción de la longitud del alineamiento y la longitud del ORF</li>
    <li>Pesencia de elementos transponibles (pero descartado en el papper, debido a que toma mucho tiempo calcular y no aportar mucho en términos del proceso de clasificación)</li>
    <li>Divergencia del transcriptoma del elemento transponible (pero descartado en el papper, debido a que toma mucho tiempo calcular y no aportar mucho en términos del proceso de clasificación)</li>
</ol>
<h2>1.1.Longitud del transcriptoma</h2>
Este feature ya se calculó.
<h2>1.2.Longitud del open reading frame (ORF)</h2>
Para calcular este feature, se utilizó en el paper el programa CPAT.<br/>
<h3>1.2.1.CPAT</h3>
Se procede con la instalación de programa.
<h4>1.2.1.1.Instalación de CPAT</h4>

In [3]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install cpat

  Using cached https://files.pythonhosted.org/packages/52/4e/63b692096ba9b3f4b399897744b88685dd68be01ff8f341f5c59fd514e34/CPAT-2.0.0-py3-none-any.whl


Una vez instalado, existe otro requisito utilizado como input por CPAT...
<h4>1.2.1.2.Generación del las tablas de frame hexamer (6mer) frequency</h4>
De acuerdo a <a href="http://rna-cpat.sourceforge.net/">http://rna-cpat.sourceforge.net/</a> para generar lo anterior, se requieren CDS de los ARN codificantes, y secuencias no codificantes. Lo segundo ya se posee, pero de lo primero solo se tienen los transcriptomas completos. Ensembl permite descargar las secuencias codificantes de estos archivos. Se procederá con su descarga.<br/>
<h5>1.2.1.2.1.Descarga de los CDS se Ensembl</h5>

In [1]:
%load_ext rpy2.ipython

In [3]:
%%time
%%R

library(RMySQL)
library(curl)
library(biomaRt)
library(stringr)

cat("Iniciando proceso...\n")

dsn_database = "tesis2"
dsn_hostname = "127.0.0.1"
dsn_port = 3306
dsn_uid = "tesis2"
dsn_pwd = "tesis2"

conn = dbConnect(MySQL(), user=dsn_uid, password=dsn_pwd, dbname=dsn_database, host=dsn_hostname)
rs = dbSendQuery(conn, "SELECT m.id_especie, m.especie FROM maestra_especies m JOIN especies_seleccionadas es ON m.especie = es.especie")
especies = dbFetch(rs)
dbClearResult(rs)

ensembl_mart=useMart("plants_mart", host="plants.ensembl.org")
lista_datasets = listDatasets(ensembl_mart)

if (nrow(especies) > 0) {
    for(i in 1:nrow(especies)) {
        tryCatch(
            expr = {
                cat("\rProcesando especie #", i, " ", especies[i,2], "                                      ", sep = "")
                la_especie = especies[i,2]
                especie = paste("^", la_especie, sep="")
                dataset = lista_datasets[which(str_detect(lista_datasets$description, especie)), 1]
                ensembl_ds = useDataset(dataset,mart=ensembl_mart)
                genes = getBM(attributes=c('ensembl_transcript_id', 'coding'), filters = 'transcript_biotype', values = 'protein_coding', mart = ensembl_ds)

                for(j in 1:nrow(genes)) {
                    if (j %% 1000 == 0) {
                        cat("\rProcesando especie #", i, " ", especies[i,2], " - procesadas ", j, " filas                     ", sep = "")
                    }
                    if (!(genes[j,2] == "Sequence unavailable" | str_length(genes[j,2]) > 65000)) {
                        query = sprintf("INSERT INTO secuencias_CDS (id_especie, cod_secuencia, coding) VALUES (%d, '%s', '%s')", especies[i,1], genes[j,1], genes[j,2])
                        dbSendQuery(conn, query)
                    }
                }
                cat("\rProcesando especie #", i, " ", especies[i,2], " - procesadas ", nrow(genes), " filas                     ", sep = "")
            },
            error = function(e) {
                cat("\rError al procesar la especie ", especies[i,2], "                                           \n", sep = "")
            }
        )
    }
}

dbDisconnect(conn)

cat("\rProceso finalizado...                                                                                       ")

Iniciando proceso...
Error al procesar la especie Arabidopsis thaliana                                           
Error al procesar la especie Brachypodium distachyon                                           
Error al procesar la especie Brassica napus                                           
Error al procesar la especie Brassica oleracea                                           
Error al procesar la especie Brassica rapa                                           
Error al procesar la especie Glycine max                                           
Error al procesar la especie Gossypium raimondii                                           
Error al procesar la especie Hordeum vulgare                                           
Error al procesar la especie Leersia perrieri                                           
Error al procesar la especie Manihot esculenta                                           
Error al procesar la especie Medicago truncatula                                    

CPU times: user 2min 42s, sys: 52.3 s, total: 3min 35s
Wall time: 2h 24min 56s


<h5>1.2.1.2.2.Procesamiento de archivos descargados manualmente porque fallaron en la descarga automática</h5>

In [5]:
%%time
import mysql.connector
import os

#método para leer los transcriptomas de un archivo fasta de ensembl
def leer_transcriptomas_ensembl(archivo, cur, id_especie):
    cabecera = ""
    transcriptoma = ""
    f = open(archivo, "r")
    for linea in f:
        if linea.startswith(">"):
            if transcriptoma != "":
                if transcriptoma != "Sequence unavailable" and len(transcriptoma) <= 65000:
                    query = "INSERT INTO secuencias_CDS (id_especie, cod_secuencia, coding) VALUES (%s, %s, %s)"
                    cur.execute(query, (id_especie, cabecera, transcriptoma))
                transcriptoma = ""
            cabecera = linea.rstrip("\n").lstrip(">")
        else:
            transcriptoma += linea.rstrip("\n")
    if transcriptoma != "" and len(transcriptoma) <= 65000:
        if transcriptoma != "Sequence unavailable":
            query = "INSERT INTO secuencias_CDS (id_especie, cod_secuencia, coding) VALUES (%s, %s, %s)"
            cur.execute(query, (id_especie, cabecera, transcriptoma))
        transcriptoma = ""

conn = mysql.connector.connect(
   host="localhost",
   database="tesis2",
   user="tesis2",
   passwd="tesis2"
)

print("Iniciando proceso...") 

cur = conn.cursor()

directory = '../Semana 05/descarga_manual_ensembl_CDS'

for filename in os.listdir(directory):
    if filename.endswith(".fasta"):
        especie = os.path.splitext(filename)[0]
        query = "SELECT COUNT(*) FROM maestra_especies m WHERE especie IN (SELECT c1.especie FROM conteo_lncrna_especie c1 LEFT JOIN conteo_lncrna_especie c2 ON c1.especie = REPLACE(c2.especie, '_', ' ') AND c1.fuente = 'Ensembl' AND c2.fuente = 'CantataDB' LEFT JOIN conteo_lncrna_especie c3 ON c1.especie = REPLACE(c3.especie, '_', ' ') AND c1.fuente = 'Ensembl' AND c3.fuente = 'GreeNC' WHERE c1.fuente = 'Ensembl' AND (c2.fuente IS NOT NULL OR c3.fuente IS NOT NULL)) AND m.especie = REPLACE(%s, '_', ' ')"
        cur.execute(query, [especie])
        result=cur.fetchone()
        if result[0] > 0:
            print("\rProcesando especie ", especie, "                                        ", end=" ")
            query = "SELECT e.id_especie FROM maestra_especies e WHERE e.especie = REPLACE(%s, '_', ' ')"
            cur.execute(query, [especie])
            id_especie=cur.fetchone()
            query = "delete from secuencias_CDS where id_especie = %s"
            cur.execute(query, [id_especie[0]])
            leer_transcriptomas_ensembl(directory + "/" + filename, cur, id_especie[0])
            conn.commit()

conn.close()

print("\rProceso finalizado...                                                                                       ")

Iniciando proceso...
Proceso finalizado...                                                                                       
CPU times: user 2min 6s, sys: 27.2 s, total: 2min 33s
Wall time: 5min 21s


<h5>1.2.1.2.3.Generación de archivos fasta con las secuencias CDS de los ARN codificantes, y los transcriptomas de los ARN no codificantes</h5>

In [10]:
%%time
import mysql.connector
import os

def generar_fasta(secuencias, especie, tipo, directory):
    t_tamanio = 80
    f = open(directory + especie + "." + tipo + ".fasta" ,"w+")
    for transcriptoma in secuencias:
        f.write(">%s\n" % (transcriptoma[0]))
        seq = transcriptoma[1]
        t_partes = [seq[i:i+t_tamanio] for i in range(0, len(seq), t_tamanio)]
        for t_parte in t_partes:
            f.write("%s\n" % (t_parte))
    f.close()

conn = mysql.connector.connect(
   host="localhost",
   database="tesis2",
   user="tesis2",
   passwd="tesis2"
)

print("Iniciando proceso...") 

cursor = conn.cursor()

directory = '../Semana 05/fasta_para_hexamer/'

query = "SELECT m.id_especie, REPLACE(m.especie, ' ', '_') FROM maestra_especies m JOIN especies_seleccionadas es ON m.especie = es.especie"
cursor.execute(query)
especies = cursor.fetchall()
for especie in especies:
    print("\rGenerando fasta CDS para " + str(especie[0]), "                                       ", end="")
    query = "select c.cod_secuencia, c.coding from secuencias_CDS c JOIN secuencias_features f ON c.id_especie = f.id_especie AND c.cod_secuencia = f.cod_secuencia WHERE c.id_especie = %s AND f.flg_pct = 1 AND f.flg_seleccionado = 1"
    cursor.execute(query, [especie[0]])
    secuencias = cursor.fetchall()
    generar_fasta(secuencias, especie[1], "CDS", directory)
    print("\rGenerando fasta lncRNA para " + str(especie[0]), "                                       ", end="")
    query = "SELECT cod_secuencia, secuencia FROM secuencias WHERE id_especie = %s AND flg_pct = 0 AND flg_seleccionado = 1"
    cursor.execute(query, [especie[0]])
    secuencias = cursor.fetchall()
    generar_fasta(secuencias, especie[1], "lncRNA", directory)

conn.close()

print("\rProceso finalizado...                                                                                       ")

Iniciando proceso...
Proceso finalizado...                                                                                       
CPU times: user 7.48 s, sys: 708 ms, total: 8.19 s
Wall time: 1min 40s


<h5>1.2.1.2.4.Generación de la tabla hexamer</h5>

In [15]:
%%time
import mysql.connector
import os
from IPython.utils import io

conn = mysql.connector.connect(
   host="localhost",
   database="tesis2",
   user="tesis2",
   passwd="tesis2"
)

print("Iniciando proceso...") 

cursor = conn.cursor()

input_directory = '../Semana 05/fasta_para_hexamer/'
output_directory = '../Semana 05/tablas_hexamer/'

query = "SELECT m.id_especie, REPLACE(m.especie, ' ', '_') FROM maestra_especies m JOIN especies_seleccionadas es ON m.especie = es.especie"
cursor.execute(query)
especies = cursor.fetchall()
for especie in especies:
    print("\rGenerando tabla hexamer para " + str(especie[0]), "                                       ", end="")
    script = "~/anaconda3/bin/make_hexamer_tab.py"
    fasta_cds = "'" + input_directory + especie[1] + ".CDS.fasta" + "'" 
    lncRNA_cds = "'" + input_directory + especie[1] + ".lncRNA.fasta" + "'"
    salida = output_directory + especie[1] + ".tsv"
    with io.capture_output() as captured:
        %run $script -c $fasta_cds -n $lncRNA_cds
        f = open(salida ,"w+")
        f.write(captured.stdout)
        f.close()

conn.close()

print("\rProceso finalizado...                                                                                       ")

Iniciando proceso...
Proceso finalizado...                                                                                       
CPU times: user 1min 11s, sys: 0 ns, total: 1min 11s
Wall time: 1min 11s


Un segundo requisito para ejecutar CPAT, será generar el archivo de regresión logística.
<h4>1.2.1.3.Generación del archivo de regresión logística</h4>
Este archivo requiere los transcriptomas de los ARN codificantes. Aún no se han generado archivos fasta para ellos.
<h5>1.2.1.3.1.Generación de archivos fasta de transcriptomas de ARN codificantes</h5>

In [17]:
%%time
import mysql.connector
import os

def generar_fasta(secuencias, especie, tipo, directory):
    t_tamanio = 80
    f = open(directory + especie + "." + tipo + ".fasta" ,"w+")
    for transcriptoma in secuencias:
        f.write(">%s\n" % (transcriptoma[0]))
        seq = transcriptoma[1]
        t_partes = [seq[i:i+t_tamanio] for i in range(0, len(seq), t_tamanio)]
        for t_parte in t_partes:
            f.write("%s\n" % (t_parte))
    f.close()

conn = mysql.connector.connect(
   host="localhost",
   database="tesis2",
   user="tesis2",
   passwd="tesis2"
)

print("Iniciando proceso...") 

cursor = conn.cursor()

directory = '../Semana 05/fasta_pct_para_logit/'

query = "SELECT m.id_especie, REPLACE(m.especie, ' ', '_') FROM maestra_especies m JOIN especies_seleccionadas es ON m.especie = es.especie"
cursor.execute(query)
especies = cursor.fetchall()
for especie in especies:
    print("\rGenerando fasta CDS para " + str(especie[0]), "                                       ", end="")
    query = "SELECT cod_secuencia, secuencia FROM secuencias WHERE id_especie = %s AND flg_pct = 1 AND flg_seleccionado = 1"
    cursor.execute(query, [especie[0]])
    secuencias = cursor.fetchall()
    generar_fasta(secuencias, especie[1], "PCT", directory)

conn.close()

print("\rProceso finalizado...                                                                                       ")

Iniciando proceso...
Proceso finalizado...                                                                                       
CPU times: user 10.1 s, sys: 1.32 s, total: 11.4 s
Wall time: 1min 55s


<h5>1.2.1.3.2.Generación del archivo de regresión logística</h5>

In [20]:
%%time
import mysql.connector
import os
from IPython.utils import io

conn = mysql.connector.connect(
   host="localhost",
   database="tesis2",
   user="tesis2",
   passwd="tesis2"
)

print("Iniciando proceso...") 

cursor = conn.cursor()

input_directory = '../Semana 05/tablas_hexamer/'
input_directory_pct = '../Semana 05/fasta_pct_para_logit/'
input_directory_lncRNA = '../Semana 05/fasta_para_hexamer/'
output_directory_logit = '../Semana 05/modelo_regresion_logistica/'

query = "SELECT m.id_especie, REPLACE(m.especie, ' ', '_') FROM maestra_especies m JOIN especies_seleccionadas es ON m.especie = es.especie"
cursor.execute(query)
especies = cursor.fetchall()
for especie in especies:
    print("\rGenerando modelo de regresión para " + str(especie[0]), "                                       ", end="")
    script = "~/anaconda3/bin/make_logitModel.py"
    archivo_hexamer = "'" + input_directory + especie[1] + ".tsv" + "'"
    fasta_pct = "'" + input_directory_pct + especie[1] + ".PCT.fasta" + "'"
    lncRNA_cds = "'" + input_directory_lncRNA + especie[1] + ".lncRNA.fasta" + "'"
    salida = "'" + output_directory_logit + especie[1] + "'"
    with io.capture_output() as captured:
        %run $script -x $archivo_hexamer -c $fasta_pct -n $lncRNA_cds -o $salida

conn.close()

print("\rProceso finalizado...                                                                                       ")

Iniciando proceso...
Proceso finalizado...                                                                                       
CPU times: user 6min 39s, sys: 0 ns, total: 6min 39s
Wall time: 6min 44s


<h4>1.2.1.2.Ejecución de CPAT</h4>

In [ ]:
%%time
import mysql.connector
import os
from IPython.utils import io

conn = mysql.connector.connect(
   host="localhost",
   database="tesis2",
   user="tesis2",
   passwd="tesis2"
)

print("Iniciando proceso...") 

cursor = conn.cursor()

input_directory = '../Semana 05/tablas_hexamer/'
input_directory_pct = '../Semana 05/fasta_pct_para_logit/'
input_directory_lncRNA = '../Semana 05/fasta_para_hexamer/'
input_directory_logit = '../Semana 05/modelo_regresion_logistica/'
output_directory = '../Semana 05/cpat_features/'

query = "SELECT m.id_especie, REPLACE(m.especie, ' ', '_') FROM maestra_especies m JOIN especies_seleccionadas es ON m.especie = es.especie"
cursor.execute(query)
especies = cursor.fetchall()
for especie in especies:
    print("\rGenerando features cpat para PTC " + str(especie[0]), "                                       ", end="")
    script = "~/anaconda3/bin/cpat.py"
    fasta_pct = "'" + input_directory_pct + especie[1] + ".PCT.fasta" + "'"
    archivo_logit = "'" + input_directory + especie[1] + "_logitModel.RData" + "'"
    archivo_hexamer = "'" + input_directory + especie[1] + ".tsv" + "'"
    salida = "'" + output_directory + especie[1] + ".PCT" + "'"
    with io.capture_output() as captured:
        %run $script -g $fasta_pct -d $archivo_logit -x $archivo_hexamer -o $salida
    print("\rGenerando features cpat para lncRNA " + str(especie[0]), "                                       ", end="")
    fasta_lncRNA = "'" + input_directory_lncRNA + especie[1] + ".lncRNA.fasta" + "'"
    salida = "'" + output_directory + especie[1] + ".lncRNA" + "'"
    #with io.capture_output() as captured:
    %run $script -g $fasta_lncRNA -d $archivo_logit -x $archivo_hexamer -o $salida

conn.close()

print("\rProceso finalizado...                                                                                       ")

Iniciando proceso...
Generando features cpat para lncRNA 2                                        

Input gene file is in FASTA format
 4208 genes finished68 genes finished1013 genes finished1371 genes finished2029 genes finished2365 genes finished3351 genes finished

Generando features cpat para lncRNA 5                                        

Input gene file is in FASTA format
 4208 genes finished69 genes finished957 genes finished1192 genes finished1467 genes finished1948 genes finished2185 genes finished2658 genes finished3156 genes finished3866 genes finished4133 genes finished

Generando features cpat para lncRNA 6                                        

...

Generando features cpat para PTC 10                                        

KeyboardInterrupt: 

Generando features cpat para lncRNA 10                                        

Input gene file is in FASTA format
 4208 genes finished62 genes finished874 genes finished1331 genes finished1781 genes finished2019 genes finished2265 genes finished2528 genes finished2945 genes finished3158 genes finished3352 genes finished3793 genes finished

Generando features cpat para PTC 11                                        

Con esto se tienen archivo de salida de CPAT con el ORF (y otros features de interés).<br/>
Se procede a la importación de estos features a la base de datos.<br/>
<h4>1.2.1.3.Importación del ORF y otros features</h4>

In [8]:
%%time 
import mysql.connector
import os
import csv

def procesar_features_cpat(archivo, cur, id_especie):
    with open(archivo, "r") as tabular:
        cpat_reader = csv.reader(tabular, delimiter=("\t"))
        for row in cpat_reader:
            cod_secuencia = row[0]
            orf_length = float(row[2])
            fickett = float(row[3])
            hexamer = float(row[4])
            query = "UPDATE secuencias_features SET orf_length = %s, orf_coverage = %s/longitud, ficket_score = %s, hexamer_score = %s WHERE id_especie = %s and cod_secuencia = %s"
            cur.execute(query, (orf_length, orf_length, fickett, hexamer, id_especie, cod_secuencia))

conn = mysql.connector.connect(
   host="localhost",
   database="tesis2",
   user="tesis2",
   passwd="tesis2"
)

print("Iniciando proceso...") 

cur = conn.cursor()

directory = '../Semana 05/cpat_features'

for filename in os.listdir(directory):
    if filename.endswith(".dat"):
        especie = filename.split(".")[0]
        query = "SELECT COUNT(*) FROM maestra_especies m WHERE especie IN (SELECT c1.especie FROM conteo_lncrna_especie c1 LEFT JOIN conteo_lncrna_especie c2 ON c1.especie = REPLACE(c2.especie, '_', ' ') AND c1.fuente = 'Ensembl' AND c2.fuente = 'CantataDB' LEFT JOIN conteo_lncrna_especie c3 ON c1.especie = REPLACE(c3.especie, '_', ' ') AND c1.fuente = 'Ensembl' AND c3.fuente = 'GreeNC' WHERE c1.fuente = 'Ensembl' AND (c2.fuente IS NOT NULL OR c3.fuente IS NOT NULL)) AND m.especie = REPLACE(%s, '_', ' ')"
        cur.execute(query, [especie])
        result=cur.fetchone()
        if result[0] > 0:
            print("\rProcesando especie", especie, filename.split(".")[1],"                         ", end=" ")
            query = "SELECT e.id_especie FROM maestra_especies e WHERE e.especie = REPLACE(%s, '_', ' ')"
            cur.execute(query, [especie])
            id_especie=cur.fetchone()
            procesar_features_cpat(directory + "/" + filename, cur, id_especie[0])
            conn.commit()

conn.close()

print("\rProceso finalizado...                                                                                       ")

Iniciando proceso...
Proceso finalizado...                                                                                       
CPU times: user 16.9 s, sys: 4.96 s, total: 21.9 s
Wall time: 1min 11s


<h2>1.3.Porcentaje de nucleótidos GC presentes</h2>
El caĺculo se realiza a través de la librería Bio.SeqUtils.<br/>

In [11]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install biopython

    100% |████████████████████████████████| 2.2MB 7.0MB/s ta 0:00:011


In [13]:
%%time
import mysql.connector
import os
from Bio.SeqUtils import GC

conn = mysql.connector.connect(
   host="localhost",
   database="tesis2",
   user="tesis2",
   passwd="tesis2"
)

print("Iniciando proceso...") 

cursor = conn.cursor()

query = "SELECT m.id_especie, REPLACE(m.especie, ' ', '_') FROM maestra_especies m JOIN especies_seleccionadas es ON m.especie = es.especie"
cursor.execute(query)
especies = cursor.fetchall()
for especie in especies:
    print("\rGenerando GC content " + str(especie[0]), "                                       ", end="")
    query = "SELECT cod_secuencia, secuencia FROM secuencias WHERE id_especie = %s AND flg_seleccionado = 1"
    cursor.execute(query, [especie[0]])
    transcriptomas = cursor.fetchall()
    for transcriptoma in transcriptomas:
        gc_content = GC(transcriptoma[1])
        query = "UPDATE secuencias_features SET gc_content = %s WHERE id_especie = %s and cod_secuencia = %s"
        cur.execute(query, (gc_content, especie[0], transcriptoma[0]))
    conn.commit()
    
conn.close()

print("\rProceso finalizado...                                                                                       ")

Iniciando proceso...
Proceso finalizado...                                                                                       
CPU times: user 21.6 s, sys: 5.04 s, total: 26.7 s
Wall time: 6min 42s


<h2>1.4.Fickett score</h2>
El caĺculo ya ha sido realizado con CPAT.<br/>
<h2>1.5.Hexamer score</h2>
El caĺculo ya ha sido realizado con CPAT.<br/>
<h2>1.6.Alignment identity in SwissProt database</h2>
Para el cálculo de este feature se usó el algoritmo diamond para la búsqueda por alineamiento.
<h3>1.6.1.Instalación de diamond</h3>

In [14]:
# Install a conda package in the current Jupyter kernel
import sys
!conda install --yes --prefix {sys.prefix} -c bioconda diamond

Solving environment: done

## Package Plan ##

  environment location: /home/jose/anaconda3

  added / updated specs:
    - diamond


The following NEW packages will be INSTALLED:

  diamond            bioconda/linux-64::diamond-0.9.24-ha87ae23_0


Preparing transaction: done
Verifying transaction: done
Executing transaction: done


Para utilizar diamond, se debe generar una base de datos de origen, en dónde se buscarán las coincidencias.
<h3>1.6.2.Base de datos diamond</h3>
Se descargó un archivo fasta con la información a ingresar a la base de datos, desde <a href="https://www.uniprot.org/uniprot/?query=taxonomy:viridiplantae&fil=reviewed%3Ayes&sort=score">https://www.uniprot.org/uniprot/?query=taxonomy:viridiplantae&fil=reviewed%3Ayes&sort=score</a><br>
Se filtraron los resultados para plantas, validadas experimentalmente.<br/>
Con esto se procede a crear la base de datos

In [23]:
%%time
import os

!~/anaconda3/bin/diamond makedb --in '../Semana 05/diamond/uniprot-viridiplantae-reviewed.fasta' -d '../Semana 05/diamond/uniprot-viridiplantae-reviewed'

diamond v0.9.24.125 | by Benjamin Buchfink <buchfink@gmail.com>
Licensed under the GNU GPL <https://www.gnu.org/licenses/gpl.txt>
Check http://github.com/bbuchfink/diamond for updates.

#CPU threads: 4
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Database file: ../Semana 05/diamond/uniprot-viridiplantae-reviewed.fasta
Opening the database file...  [0.000237s]
Loading sequences...  [0.115702s]
Masking sequences...  [2.24115s]
Writing sequences...  [0.040182s]
Hashing sequences...  [0.009778s]
Loading sequences...  [2.5e-05s]
Writing trailer...  [0.001172s]
Closing the input file...  [2.6e-05s]
Closing the database file...  [3.8e-05s]
Database hash = e3fb30fb072672cc3aab0b36faa2ef6d
Processed 39800 sequences, 15258335 letters.
Total time = 2.40875s
CPU times: user 41.9 ms, sys: 16 ms, total: 57.8 ms
Wall time: 2.54 s


<h3>1.6.3.Búsqueda de alineamientos con diamond</h3>
Para la búsqueda se utilizará un comando com parámetros similares a los del papper.

In [1]:
%%time
import mysql.connector
import os
from IPython.utils import io

print("Iniciando proceso...") 

cursor = conn.cursor()

input_directory_pct = '../Semana 05/fasta_pct_para_logit/'
input_directory_lncRNA = '../Semana 05/fasta_para_hexamer/'
output_directory = '../Semana 05/diamond_features/'

query = "SELECT m.id_especie, REPLACE(m.especie, ' ', '_') FROM maestra_especies m JOIN especies_seleccionadas es ON m.especie = es.especie"
cursor.execute(query)
especies = cursor.fetchall()
for especie in especies:
    print("\rRealizando búsqueda para " + str(especie[0]), " PCT                                      ", end="")
    diamond_db = "'" + '../Semana 05/diamond/uniprot-viridiplantae-reviewed.dmnd' + "'"
    fasta_pct = "'" + input_directory_pct + especie[1] + ".PCT.fasta" + "'"
    lncRNA_cds = "'" + input_directory_lncRNA + especie[1] + ".lncRNA.fasta" + "'"
    salida = "'" + output_directory + especie[1] + ".PCT.tsv" + "'"
    with io.capture_output() as captured:
        !~/anaconda3/bin/diamond blastx -d $diamond_db -q $fasta_pct -o $salida -k 5 --gapopen 11 --gapextend 1 --more-sensitive -f 6 qseqid pident length qframe qstart qend sstart send evalue bitscore
    print("\rRealizando búsqueda para " + str(especie[0]), " lncRNA                                      ", end="")
    lncRNA_cds = "'" + input_directory_lncRNA + especie[1] + ".lncRNA.fasta" + "'"
    salida = "'" + output_directory + especie[1] + ".lncRNA.tsv" + "'"
    with io.capture_output() as captured:
        !~/anaconda3/bin/diamond blastx -d $diamond_db -q $lncRNA_cds -o $salida -k 5 --gapopen 11 --gapextend 1 --more-sensitive -f 6 qseqid pident length qframe qstart qend sstart send evalue bitscore
    
conn.close()

print("\rProceso finalizado...                                                                                       ")

Iniciando proceso...
Proceso finalizado...                                                                                       
CPU times: user 13.8 s, sys: 2.25 s, total: 16.1 s
Wall time: 51min 51s


<h3>1.6.4.Importación de features a la base de datos</h3>

In [16]:
%%time

import mysql.connector
import csv
import os

conn = mysql.connector.connect(
   host="localhost",
   database="tesis2",
   user="tesis2",
   passwd="tesis2"
)

#adaptado de https://github.com/gbgolding/crema/blob/master/bin/featuresetup_module.py
def transcript_info_dict(id_especie, blast_file, cursor, flg_pct):
    transcript_dict = {}
    with open(blast_file, "r") as f:
        tab_reader = csv.reader(f, delimiter=("\t"))
        line_1 = next(tab_reader)
        first = line_1[0]
        score = [float(line_1[9])]
        with_len = [[first, float(line_1[1]), float(line_1[2]), float(line_1[3]), float(line_1[9])]] # name identity length frame score
        for row in tab_reader:
            if row[0] == first:
                score.append(float(row[9]))
                with_len.append([row[0], float(row[1]), float(row[2]), float(row[3]), float(row[9])])
            else:
                transcript_dict[first] = {}
                transcript_dict[first]["identity"] = float(0)
                transcript_dict[first]["align_length"] = float(0)
                max_value = max(score)
                max_index = score.index(max_value)
                max_len_ident = with_len[max_index]
                if max_len_ident[3] > 0:
                    transcript_dict[first]["identity"] = float(max_len_ident[1])
                    transcript_dict[first]["align_length"] = float(max_len_ident[2])
                score = [float(row[9])]
                first = row[0]
                with_len = [[first, float(row[1]), float(row[2]), float(row[3]), float(row[9])]]
        transcript_dict[first] = {}
        transcript_dict[first]["identity"] = float(0)
        transcript_dict[first]["align_length"] = float(0)
        max_value = max(score)
        max_index = score.index(max_value)
        max_len_ident = with_len[max_index]
        if max_len_ident[3] > 0:
            transcript_dict[first]["identity"] = float(max_len_ident[1])
            transcript_dict[first]["align_length"] = float(max_len_ident[2])
    #fin de código adaptado de https://github.com/gbgolding/crema/blob/master/bin/featuresetup_module.py
    query = "SELECT cod_secuencia, IFNULL(longitud, 0), IFNULL(orf_length, 0) FROM secuencias_features WHERE flg_seleccionado = 1 AND id_especie = %s AND flg_pct = %s"
    cursor.execute(query, (id_especie, flg_pct))
    transcriptomas = cursor.fetchall()
    for transcriptoma in transcriptomas:
        cod_secuencia = transcriptoma[0]
        longitud = transcriptoma[1]
        orf_length = transcriptoma[2]
        if transcriptoma[0] in transcript_dict:
            transcript_dict[cod_secuencia]["align_perc_len"] = float(transcript_dict[cod_secuencia]["align_length"]/longitud)
            if orf_length == 0:
                transcript_dict[cod_secuencia]["align_perc_ORF"] = float(0)
            else:
                transcript_dict[cod_secuencia]["align_perc_ORF"] = float(transcript_dict[cod_secuencia]["align_length"]/orf_length)
        else:
            transcript_dict[cod_secuencia] = {}
            transcript_dict[cod_secuencia]["identity"] = float(0)
            transcript_dict[cod_secuencia]["align_length"] = float(0)
            transcript_dict[cod_secuencia]["align_perc_len"] = float(0)
            transcript_dict[cod_secuencia]["align_perc_ORF"] = float(0)
        query = "UPDATE secuencias_features SET alignment = %s, alignment_length = %s, alignment_proportion = %s, alignment_proportion_orf = %s WHERE id_especie = %s AND cod_secuencia = %s"
        cursor.execute(query, (transcript_dict[cod_secuencia]["identity"], transcript_dict[cod_secuencia]["align_length"], transcript_dict[cod_secuencia]["align_perc_len"], transcript_dict[cod_secuencia]["align_perc_ORF"], id_especie, cod_secuencia))

print("Iniciando proceso...") 

cursor = conn.cursor()

output_directory = '../Semana 05/diamond_features/'

query = "SELECT m.id_especie, REPLACE(m.especie, ' ', '_') FROM maestra_especies m JOIN especies_seleccionadas es ON m.especie = es.especie"
cursor.execute(query)
especies = cursor.fetchall()
for especie in especies:
    print("\rExportando features para " + str(especie[0]), " PCT                                      ", end="")
    salida = output_directory + especie[1] + ".PCT.tsv"
    transcript_info_dict(especie[0], salida, cursor, 1)
    conn.commit()
    print("\rExportando features para " + str(especie[0]), " lncRNA                                      ", end="")
    salida = output_directory + especie[1] + ".lncRNA.tsv"
    transcript_info_dict(especie[0], salida, cursor, 0)
    conn.commit()
    
conn.close()

print("\rProceso finalizado...                                                                                       ")

Iniciando proceso...
Proceso finalizado...                                                                                       
CPU times: user 16.8 s, sys: 4.41 s, total: 21.2 s
Wall time: 57.7 s


<h2>1.7.Longitud del alineamiento en la base de datos SwissProt</h2>
Ya se proceso con el algoritmo diamond.<br>
<h2>1.8.Proporción de la longitud del alineamiento con respecto a la longitud del transcriptoma</h2>
Ya se proceso con el algoritmo diamond.<br>
<h2>1.9.Proporción de la longitud del alineamiento y la longitud del ORF</h2>
Ya se proceso con el algoritmo diamond.<br>